In [1]:
from pathlib import Path
import sys
project_root = Path().resolve().parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Added {project_root} to sys.path")

# sys.path.append(str(project_root / "src"))

In [2]:
from ai_engineer.generate_ideas import ChatManager
import os

import logging

# Configure logging with both console and file output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # Console output
        logging.FileHandler('trylog.log')  # File output
    ]
)

chat_manager = ChatManager(
        problem="What is the best project to build for the hackathon?",
        # docs_path="examples/hackathon01/resource/document/aggregated.md",
        model="gpt-4o-mini",
        api_key=os.environ["OPENAI_API_KEY"],
    )
chat_manager.rag_chat()

2024-11-06 12:55:29,641 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
/Users/nvtrang/Library/Caches/pypoetry/virtualenvs/ai-engineer-ooiPA4kL-py3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-11-06 12:55:31,374 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-11-06 12:55:35,943 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-06 12:55:35,997 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `groupchat`.
2024-11-06 12:55:36,

Trying to create collection.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

: 

In [ ]:
import polars as pl
import json

def parse_log_line(line):
    """Parse a single log line into a dictionary."""
    try:
        # Try to parse as JSON
        return json.loads(line)
    except json.JSONDecodeError:
        # For non-JSON lines (like "Started new session...")
        if line.startswith("Started new session"):
            return {
                "event_type": "session_start",
                "message": line.strip()
            }
        elif line.startswith("[file_logger]"):
            return {
                "event_type": "file_logger",
                "message": line.strip()[13:]  # Remove "[file_logger] " prefix
            }
        return None

def load_autogen_logs(file_path):
    """Load autogen logs into a Polar DataFrame."""
    # Read all lines from the file
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Parse each line into a dictionary
    parsed_data = []
    for line in lines:
        line = line.strip()
        if not line:  # Skip empty lines
            continue
            
        parsed = parse_log_line(line)
        if parsed:
            parsed_data.append(parsed)
    
    # Convert to Polars DataFrame
    df = pl.DataFrame(parsed_data)
    return df

# Usage
# if __name__ == "__main__":
log_file = project_root / "autogen_logs/runtime.log"  # Adjust path as needed
df = load_autogen_logs(log_file)

# Print basic information about the DataFrame
print("\nDataFrame Info:")
print(f"Shape: {df.shape}")
print("\nColumns:", df.columns)

# Print first few rows
print("\nFirst few rows:")
print(df.head())

In [ ]:
# df[['event_type', 'message', 'id',]]
for message in df['message']:
    print(message)


In [ ]:
import sqlite3

# Connect to the SQLite database
db_path = project_root / "tmp/db/chroma.sqlite3"  # Adjust path as needed
conn = sqlite3.connect(db_path)

# Read data from the database
# Get list of all tables in the database
# query_tables = "SELECT name FROM sqlite_master WHERE type='table'"
# tables = pl.read_database(query=query_tables, connection=conn)
# print("\nAvailable tables in database:")
# print(tables)

query = "SELECT * FROM segment_metadata"  # Adjust table name if different
df_sqlite = pl.read_database(query=query, connection=conn)

# Print basic information about the DataFrame from SQLite
print("\nSQLite DataFrame Info:")
print(f"Shape: {df_sqlite.shape}")
print("\nColumns:", df_sqlite.columns)

# Print first few rows
print("\nFirst few rows from SQLite:")
print(df_sqlite.head())

# Close the connection
conn.close()


In [ ]:

for name in tables['name']:
    print(name)
